#版本说明
- ver2.10:增加5.6.1
- ver2.9:去掉app_version异常数据
- ver2.8:增加配件(5)
- ver2.7:增加5.6版本
- ver2.6:xlsx改成csv,No engine for filetype: 'csv',看来要加engine
- ver2.5:修正uv合计，之前的都不正确，uv不能求和（无意义，涉及到去重）——未完成
- ver2.4:需要增加如果超过一万行，则hit>10,处理搜索关键词过多，以前只在月数据中使用，修改了tidy
- ver2.3:常规使用不必修改时间；非常规使用需要修改时间，同时可能需要修改table
- ver2.2:[1,3,4]字符化['1','3','4']


In [3]:
import pymysql
import pandas as pd
import numpy as np
import time
from pandas import ExcelWriter
import os

db='gather_data'
mydb = pymysql.connect(host="ip",
                       port=3300,
                       user="root",
                       passwd="Ctyun001~",
                       db=db,
                       charset='utf8'
                     )

# 以下两块为可能需要修改的地方

In [5]:
table='day_gather'

#用于临时选择时间，非常规
startDate = "'2016-07-11'"
endDate = "'2016-07-17'"

# global variables

In [6]:
starttime=time.time()
starttime

1471916196.7398002

In [7]:
import datetime
today=datetime.date.today()
start=today-datetime.timedelta(days=datetime.date.isoweekday(today)+6)
startDate="'{}'".format(str(start))
end=today-datetime.timedelta(days=datetime.date.isoweekday(today))
endDate="'{}'".format(str(end))

In [8]:
startDate

"'2016-08-15'"

In [9]:
endDate

"'2016-08-21'"

In [10]:
#这里面没有sheet‘汇总’
sheetname_list=['支付', '首页','更多', '导航按钮','查询办理' ,'搜索关键词', '我的' ]
#app_version_4=['5.5.0', '5.4.0', '5.3.0', '5.2.0']

date_type = "'dayVCT'"

In [11]:
app_version_list=['5.6.1','5.6.0','5.5.0', '5.4.0', '5.3.0', '5.2.0',['5.6.1','5.6.0','5.5.0', '5.4.0', '5.3.0', '5.2.0'],
                  ['5.0.0',
 '5.0.0(F)',
 '5.1.0(F)',
 '5.1.5(F)',
 '5.2.0',
 '5.3.0',
 '5.4.0',
 '5.5.0',
 '5.5.1',
 '5.6.0',
 '5.6.1']]

In [12]:
timeinterval='({}-{})'.format(''.join(startDate.split('-'))[1:-1],''.join(endDate.split('-'))[1:-1])

In [13]:
timeinterval

'(20160815-20160821)'

In [14]:
folder='andriodweek'+timeinterval

if not os.path.exists(folder):
    os.mkdir(folder)

# functions

In [15]:
def to_sqls(channel_name = None,
            date_type = "'dayVCT'",
            startDate = "'2016-06-06'",
            endDate = "'2016-06-12'",
            app_version = None,
            table ='day_gather',
            product_category = None):
    '''app_version="'5.2.0'"'''
    
    sql='select * from {} where type={} and create_date>={} and create_date<={}'\
    .format(table,date_type,startDate,endDate)
    if not channel_name is None:
        sql+=' and channel_name={}'.format(channel_name)
    if not app_version is None:
        sql+=' and app_version in {}'.format(app_version)
    if not product_category is None :
        sql+=' and product_category ={}'.format(product_category)
    
    return sql

In [16]:
def getto_df(sql):
    return pd.read_sql(sql,con=mydb)

In [17]:
def df_tidy(df,index_name='module',channel_name=None):
    '''
    index_name='module'
    调整reshape data
    添加合计
    '''
    df1=df.iloc[:,1:].groupby([index_name,'create_date']).sum().unstack(1)
    df=df1.sort_values(by=df1.columns[0],ascending=False)
    
    # 去掉index中过长字符列（通常是异常数据）
    #df=df2.loc[list(map(right_len,df2.index))]
    # 列名译成中文,
    df=tranto_c_df(df)
    # 去掉含'�'的行——这一行会提示warning，好像并不是，报错消失了
    #df=df.loc[list(map(lambda x:'�' not in x,df.index))]
    # 添加合计，此合计uv有误，汇总有误
    df=add_sum(df)
    #处理搜索关键词过多
    if len(df)>10000 and channel_name== "'搜索关键词'":
       df=df[df>10].dropna()
    
    return df

In [18]:
def tranto_c_df(df):
    df1=df.stack()

    df1.columns=tranto_c(df.stack().columns)

    df2=df1.unstack()
    
    return df2

def tranto_c(col,
            e_col=['hits','hit','order_confirms', 'order_payment', 'product_details','visitors','visitor'],
            c_col=['点击量','点击量','订单确认','订单支付','商品详情','访客量','访客量']):
    '''翻译机
    以下是字典
    e_col中不能有相同元
    '''
    
    temp=[]
    for s in col:
        if s in e_col:
            temp.append(c_col[e_col.index(s)])
        else:
            temp.append(s)
    return temp

In [19]:
def right_len(s,size=18):
    '''type(s):string'''
    if len(s)<size:
        return True
    else:
        return False

In [20]:
def add_sum(df):
    '''
    添加合计'''
    df1=df.groupby(level=0,axis=1).sum()

    for s in df.stack().columns:
        df[(s,'合计')]=df1[s]

    df2=df.stack().unstack()

    df2.loc['合计',:]=df2.sum()

    df3=df2.sort_values(by=df2.columns[0],ascending=False)
    
    return df3

In [21]:
def to_tuple_sql(app_version):
    if type(app_version) == str:
        temp=list([app_version])
        temp.append('all')
        return tuple(temp)
    if type(app_version) == list:
        return tuple(app_version)

In [22]:
def to_excel(timeinterval,sheetname_list=sheetname_list,folder=folder,app_version=None):
    '''仅安卓可用
    可以用可变参数把app_version 和app_version_list统一起来
    app_version_list="('5.5.0','5.4.0','5.3.0','5.2.0')"
    appversion= "'5.5.0'" or "('5.5.0','5.4.0','5.3.0','5.2.0')"
    string,list,None(all versions)
    '''
    #list to_string "'5.5.0'" or "('5.5.0','5.4.0','5.3.0','5.2.0')"
    if app_version is None:
        pathnew=folder+'/'+'andriod'+"'all'"+timeinterval+'.xlsx'
        dict_sum={}
        with ExcelWriter(pathnew,datetime_format='yyyy/mm/dd') as writer:
            for s in sheetname_list:
                if s == '汇总':
                    channel_name= None
                else:
                    channel_name="'{}'".format(s)

                sql=to_sqls(channel_name = channel_name,
                            date_type = date_type,
                            startDate =startDate,
                            endDate = endDate,
                            app_version = app_version,
                            table=table)
                df=df_tidy(getto_df(sql),channel_name=channel_name)
                dict_sum[s]=df.iloc[0]
                df.to_excel(writer,sheet_name=s)
            #汇总sheet
            df_sum=pd.DataFrame(dict_sum)
            df_sum.T.to_excel(writer,sheet_name='汇总',index_label='module')
                
                
    if not app_version is None:
        pathnew=folder+'/'+'andriod'+str(app_version)+timeinterval+'.xlsx'
        dict_sum={}
        with ExcelWriter(pathnew,datetime_format='yyyy/mm/dd') as writer:
            for s in sheetname_list:
                if s == '汇总':
                    channel_name= None
                else:
                    channel_name="'{}'".format(s)

                sql=to_sqls(channel_name = channel_name,
                            date_type = date_type,
                            startDate =startDate,
                            endDate = endDate,
                            app_version = "{}".format(to_tuple_sql(app_version)),
                            table=table)
                df=df_tidy(getto_df(sql),channel_name=channel_name)
                dict_sum[s]=df.iloc[0]
                df.to_excel(writer,sheet_name=s)
            #汇总sheet
            df_sum=pd.DataFrame(dict_sum)
            df_sum.T.to_excel(writer,sheet_name='汇总',index_label='module')

In [23]:
for s in app_version_list:
    to_excel(timeinterval,sheetname_list=sheetname_list,folder=folder,app_version=s)

# 商品数据

In [24]:
pathnew=folder+'/'+'andriod'+'商品数据'+timeinterval+'.xlsx'

writer=ExcelWriter(pathnew,datetime_format='yyyy/mm/dd')

table = 's_product_gather'

In [25]:
product_category_list=[1,3,4,5]
p_c_list=['手机','套餐','号码','配件']

In [26]:
date_type = "'dayCP'"
for i,s in enumerate(product_category_list):
    sql=to_sqls(date_type = date_type,
                startDate =startDate,
                endDate = endDate,
                table=table,
                product_category = s)
    df=df_tidy(getto_df(sql),index_name='product')
    df.to_excel(writer,sheet_name=p_c_list[i])

In [27]:
    
date_type = "'dayCT'"
sql=to_sqls(date_type = date_type,
            startDate =startDate,
            endDate = endDate,
            table=table,
            )
df=df_tidy(getto_df(sql),index_name='product_category')
#去掉index 0,5
df=df.iloc[0:4,:]  
# create a new dataframe
newindex=pd.Index(tranto_c(df.index,[str(x) for x in product_category_list],p_c_list),name=df.index.name)
df1=pd.DataFrame(df.values,index=newindex,columns=df.columns)
df1.to_excel(writer,sheet_name='汇总')

In [28]:
date_type ="'dayS'"
sql=to_sqls(date_type = date_type,
            startDate =startDate,
            endDate = endDate,
            table=table,
            )
df=df_tidy(getto_df(sql),index_name='product')
df.to_excel(writer,sheet_name='商品优惠')

writer.save()

In [29]:
endtime=time.time()
endtime-starttime

129.47551107406616